In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import os
import random
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.applications import MobileNetV2
print(os.getcwd())

In [ ]:
#paths
csv_path = 'drive/MyDrive/Colab Notebooks/IIIP/Testing chamber/seen.csv'
seen_folder_path = 'drive/MyDrive/Colab Notebooks/IIIP/Testing chamber/plantvillagesmall/seen'
image_classifier_model = 'drive/MyDrive/Colab Notebooks/IIIP/Testing chamber/Models/image_classifier.hdf5'
image_generator = 'drive/MyDrive/Colab Notebooks/IIIP/Testing chamber/Models/image_gen.hdf5'

In [ ]:
folders = os.listdir(seen_folder_path)
print(folders)

df = pd.read_csv(csv_path, header = None)
print(df)

In [ ]:
# make dataset
seen_dataset = []
combined_label_dict = {}

In [ ]:
for index,row in df.iterrows():
  curr_folder_path = seen_folder_path + "/" + str(row[0])
  images = os.listdir(curr_folder_path)
  combined_label_dict[row[3]] = str(row[0])
  for i in tqdm(range(0,500)):
      select = random.choice(images)
      image = cv2.imread(os.path.join(curr_folder_path, select))
      image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image2 = tf.image.resize(image2, (128, 128))
      seen_dataset.append((image2, row[3]))
      images.remove(select)
      if (len(images) == 0):
        break

In [ ]:
print(len(seen_dataset))

In [ ]:
np.random.shuffle(plant_dataset_real)

train, test = train_test_split(plant_dataset_real, test_size = 0.4)

train_images = np.array([item[0] for item in train])
train_labels = np.array([item[1] for item in train])

test_images = np.array([item[0] for item in test])
test_labels = np.array([item[1] for item in test])

print("Train length: ", str(len(train_images)))
print("Test length: ", str(len(test_images)))

train_images = tf.image.resize(train_images, (128, 128))
train_labels = tf.one_hot(train_labels, depth=6)

test_images = tf.image.resize(test_images, (128, 128))
test_labels = tf.one_hot(test_labels, depth=6)

print(train_images.shape)
print(test_images.shape)

In [ ]:
image_input = tf.keras.layers.Input(shape=(128,128,3))

mobilenet = MobileNetV2(include_top=False, input_tensor=image_input)
mobilenet.trainable = True

image_classfier = tf.keras.Sequential([
  tf.keras.layers.Input(shape = (128,128,3)),
  mobilenet,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(6,activation='softmax')
])

image_classfier.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
image_classfier.summary()

In [ ]:
# train the model
history = model_fit = plant_image_classfierimage_real.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# evaluate model
_, train_acc = image_classfier.evaluate(train_images, train_labels)
_, test_acc = image_classfier.evaluate(test_images, test_labels )

print(f'\nTrain accuracy: {train_acc:.0%}')
print(f'Test accuracy: {test_acc:.0%}')

image_classfier.save(image_classifier_model)


In [ ]:
from tensorflow.keras import layers
latent_dim = 100
classes = 6
def define_generator():
    noise = layers.Input(shape=(latent_dim,))
    label = layers.Input(shape=(1,))

    label_embedding = layers.Flatten()(layers.Embedding(classes, latent_dim)(label))
    model_input = layers.Concatenate()([noise, label_embedding])

    x = layers.Dense(512 * 4 * 4)(model_input)

    x = layers.Reshape((4, 4, 512))(x)
    x = layers.BatchNormalization(momentum=0.9)(x)
    x = layers.LeakyReLU(0.1)(x)

    x = layers.Conv2DTranspose(64 * 8, kernel_size=4, strides=2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(), name='conv_transpose_1')(x)
    x = layers.BatchNormalization(momentum=0.1, name='bn_1')(x)
    x = layers.LeakyReLU(name='leaky_relu_1')(x)

    x = layers.Conv2DTranspose(64 * 4, kernel_size=4, strides=2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(), name='conv_transpose_2')(x)
    x = layers.BatchNormalization(momentum=0.1, name='bn_2')(x)
    x = layers.LeakyReLU(name='leaky_relu_2')(x)

    x = layers.Conv2DTranspose(64 * 2, kernel_size=4, strides=2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(), name='conv_transpose_3')(x)
    x = layers.BatchNormalization(momentum=0.1, name='bn_3')(x)
    x = layers.LeakyReLU(name='leaky_relu_3')(x)

    x = layers.Conv2DTranspose(64 * 1, kernel_size=4, strides=2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(), name='conv_transpose_4')(x)
    x = layers.BatchNormalization(momentum=0.1, name='bn_4')(x)
    x = layers.LeakyReLU(name='leaky_relu_4')(x)

    out_layer = layers.Conv2DTranspose(3, 4, 2,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(),  activation='tanh', name='conv_transpose_6')(x)

   # define model
    model = tf.keras.Model([noise, label], out_layer)
    return model

image_gen = define_generator()
image_gen.load_weights(image_generator)
print(image_gen.summary())

In [ ]:
fake_dataset = []
fake_label = []
seed = tf.random.normal([20, latent_dim])
seed.dtype
for i in range(6):
   curr_label = []
   for j in range(20):
        curr_label.append(i)
        fake_label.append(i)
   curr_label = tf.convert_to_tensor(curr_label)
   images = image_gen([seed, curr_label], training=False)
   for k in range(images.shape[0]):
      pred = (images[i, :, :, :] + 1 ) * 127.5
      pred = np.array(pred)
      fake_dataset.append(pred)

print(len(fake_dataset))

In [ ]:
real_dataset=[]
real_label=[]
for index,row in df.iterrows():
  curr_folder_path = seen_folder_path + "/" + str(row[0])
  images = os.listdir(curr_folder_path)
  combined_label_dict[row[3]] = str(row[0])
  for i in tqdm(range(0,20)):
      image = cv2.imread(os.path.join(curr_folder_path, select))
      image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image2 = tf.image.resize(image2, (128, 128))
      real_dataset.append(image2)
      real_label.append(row[3])

In [ ]:
for n in range(1,7):
    fake_class_images = []
    real_class_images = []
    fake_labels = []
    real_labels = []
    for i in range((n*20)-20, (n*20)):
        fake_class_embeddings.append(fake_dataset[i])
    for i in range((n*20)-20, (n*20)):
        real_class_embeddings.append(real_dataset[i])
        real_labels.append(eval_labels_real[i])
    for i in range(20):
        fake_overall_labels.append(n-1)
        fake_labels.append(n-1)

    fake_class_images = np.array(fake_class_images)
    real_class_images = np.array(real_class_images)

    fake_labels = tf.one_hot(np.array(fake_labels), depth=6)
    real_labels = tf.one_hot(np.array(real_labels), depth=6)


    print("=======================================")
    print("Class-wise Accuracy: Class " + str(n))
    loss, racc = embed_classifier.evaluate(real_class_images, real_labels)
    loss, facc = embed_classifier.evaluate(fake_class_images, fake_labels)
    print("Real class Accuracy: " + str(racc))
    print("Fake class Accuracy: " + str(facc))

print("=======================================")
print("Overall Accuracy:")
fake_overall_labels = tf.one_hot(np.array(fake_overall_labels), depth=6)
eval_labels_real = tf.one_hot(np.array(eval_labels_real), depth=6)

loss, racc = embed_classifier.evaluate(real_embeddings, eval_labels_real)
loss, facc = embed_classifier.evaluate(fake_embeddings, fake_overall_labels)
print("Real Overall Accuracy: " + str(racc))
print("Fake Overall Accuracy: " + str(facc))